In [1]:
import pandas as pd
import pprint as pp

In [2]:
data_dir = './data/'

In [3]:
location = pd.read_pickle(data_dir + 'mediumLocation.pickle')

In [4]:
def take(n, dict_obj):
    new_dict = {}
    count = 1
    for key,val in dict_obj.items():
        new_dict[key] = val
        count += 1
        if (count > n) : break
    return new_dict

In [5]:
pp.pprint(take(12, location))

{'U1313': ['norfolk virginia area'],
 'U14078': ['stockholm sweden'],
 'U1417': ['baltimore maryland area'],
 'U22859': ['bengaluru area india'],
 'U27476': ['greater new york city area'],
 'U27759': ['urbana-champaign illinois area'],
 'U4665': ['greater chicago area'],
 'U6021': ['urbana-champaign illinois area'],
 'U7310': ['greater seattle area'],
 'U8804': ['bologna area italy'],
 'U9628': ['greater los angeles area'],
 'U9721': ['urbana-champaign illinois area']}


In [6]:
employer = pd.read_pickle(data_dir + 'mediumEmployer.pickle')
college = pd.read_pickle(data_dir + 'mediumCollege.pickle')

In [7]:
pp.pprint(take(5, employer))
print('\n')
pp.pprint(take(5, college))

{'U1313': ['discovery education',
           'ctb mcgraw-hill',
           'university of charleston university of south carolina',
           'south carolina department of education',
           'chesapeake va and sumter sc',
           'virgina state corrections'],
 'U21998': ['illinois college advising corps',
            'victoria amplifiers',
            'university of illinois at chicago'],
 'U27476': ['ibm',
            'nyse euronext',
            'skim.me',
            'shanghai pudong development bank',
            'qad'],
 'U4665': ['technology services group - chicago', 'ge healthcare'],
 'U8804': ['gruppo euris spa',
           'iconsulting',
           'econoetica',
           'universit&#xe0; di bologna',
           'university of illinois at urbana-champaign']}


{'U1313': ['longwood university'],
 'U22041': ['university of illinois at urbana-champaign'],
 'U27476': ['shanghai jiao tong university'],
 'U4665': ['university of illinois at urbana-champaign'],
 'U8804': ['

In [39]:
empty_str = '_60percent_of_empty_profile'
get_empty = True
location_empty = pd.read_pickle(data_dir + 'mediumLocation' + (empty_str if get_empty else '') + '.pickle')
employer_empty = pd.read_pickle(data_dir + 'mediumEmployer' + (empty_str if get_empty else '') + '.pickle')
college_empty = pd.read_pickle(data_dir + 'mediumCollege' + (empty_str if get_empty else '') + '.pickle')
print(take(2, location_empty))
print('here') if 'U2747' in location_empty else print('non')

{'U1313': ['norfolk virginia area'], 'U8804': ['bologna area italy']}
non


In [40]:
print(len(location))
print(len(location_empty))
print(take(2, location))

811
336
{'U27476': ['greater new york city area'], 'U4665': ['greater chicago area']}


In [41]:
# are all 1 single element arrays ? Location yes, employers and college no
for key, lieu in location.items():
    if len(lieu) > 1 : print(lieu)

In [42]:
import networkx as nx

In [43]:
graph = nx.read_gexf(data_dir + 'mediumLinkedin.gexf')

In [13]:
#com = nx.communicability(graph)
#com

In [14]:
def getNodes(G):
    for node in G.nodes:
        print(node)
        
      

In [15]:
# --- NOT NECESSARY ---
def convertToDictList(dict_of_lists):
    new_dict = {}
    for key, arr in dict_of_lists.items():
        new_dict[key] = { key: val for key, val in enumerate(arr) } # we neeed a dictionary
    return new_dict

location_empty_dict = convertToDictList(location_empty)
college_empty_dict = convertToDictList(college_empty)
employer_empty_dict = convertToDictList(employer_empty)

In [44]:
def addAttributes(G, attributes, attributes_name):
    nx.set_node_attributes(G, attributes, attributes_name)
        
addAttributes(graph, location_empty, 'location')
addAttributes(graph, employer_empty, 'employer')
addAttributes(graph, college_empty, 'college')

In [45]:
# find a place to start (where all are well labelized)
possible_attributes = ['location', 'college', 'employer']

def checkIfAttributes(node, attributes):
    for att in attributes:
        if not att in node: return False
    return True

print(graph.nodes['U1313'])
print(checkIfAttributes(graph.nodes['U1313'], possible_attributes))

def checkWellLabeledNodes(G):
    well_labeled_nodes = []
    for node in G.nodes:
        if not checkIfAttributes(graph.nodes[node], possible_attributes): continue
        neighbors = [n for n in graph.neighbors(node)]
        completed = True
        for neighbor in neighbors:
            if not checkIfAttributes(graph.nodes[neighbor], possible_attributes): 
                completed = False
                break
        if not completed : continue
        #print('---paso')
        #print(node)
        #print(neighbors)
        # passed both tests, good places to start:
        well_labeled_nodes.append(node)
    return well_labeled_nodes

well_labeled_nodes = checkWellLabeledNodes(graph)
print(well_labeled_nodes)

{'label': 'U1313', 'location': ['norfolk virginia area'], 'employer': ['discovery education', 'ctb mcgraw-hill', 'university of charleston university of south carolina', 'south carolina department of education', 'chesapeake va and sumter sc', 'virgina state corrections'], 'college': ['longwood university']}
True
['U22859', 'U25619', 'U1441', 'U16132', 'U9130', 'U1220', 'U1068', 'U1252', 'U24117', 'U9655', 'U1196', 'U9689', 'U15994', 'U15917', 'U22843', 'U1104', 'U1199', 'U16204', 'U25641', 'U1238', 'U1147', 'U18983', 'U14131', 'U4542', 'U16153', 'U14150', 'U7402']


In [46]:
from operator import itemgetter, attrgetter

In [47]:
# Understand neighbors
main = 'U9689'
neighbors = [n for n in graph.neighbors(main)]
print('neighbors')
print(neighbors)

def searchRelationShips(G, node_1, node_2, attributes):
    common_att = {}
    node_1_dict = G.nodes[node_1]
    node_2_dict = G.nodes[node_2]
    for att in attributes:
        node_1_att_values = node_1_dict[att]
        node_2_att_values = node_2_dict[att]
        for val in node_1_att_values:
            if val in node_2_att_values: 
                if not att in common_att: common_att[att] = []
                common_att[att].append(val)
        # what to do with the found relationships
    return common_att
            

print('next')
# assuming the neighbors are well defined at first
relationships = []
for n in neighbors:
    first_level_neighbors = [n for n in graph.neighbors(n)] # convert to list in order to be able to use its properties
    first_level_neighbors.remove(main) #ignore central node
    related = [f_n for f_n in first_level_neighbors if f_n in neighbors]
    print('related')
    print(related)
    relationship = {}
    connections = {}
    sum_of_inner = 0
    sum_of_inner_len = 0
    for ele in related:
        connections[ele] = searchRelationShips(graph, ele, n, possible_attributes)
        sum_of_inner += sum([len(values) for connection, values in connections[ele].items()])
        sum_of_inner_len += len(connections[ele])
    relationship['connections'] = connections
    relationship['connections_len'] = len(connections)
    relationship['length'] = len(connections)
    relationship['inner_length_sum'] = sum_of_inner
    relationship['inner_global_sum'] = sum_of_inner_len
    relationship['from'] = n
    relationships.append(relationship)

#relationships = sorted(relationships, key=itemgetter('length'))
pp.pprint(relationships)
print(college_empty['U9660'])
print(college_empty['U9657'])
print(college_empty['U9709'])
print(college_empty['U9711'])
print(college_empty['U9689'])

neighbors
['U9660', 'U9657', 'U9709', 'U9711']
next
related
['U9709', 'U9711', 'U9657']
related
['U9660']
related
['U9660']
related
['U9660']
[{'connections': {'U9657': {'college': ['university of illinois at '
                                        'urbana-champaign'],
                            'employer': ['university of illinois at '
                                         'urbana-champaign'],
                            'location': ['urbana-champaign illinois area']},
                  'U9709': {'college': ['university of illinois at '
                                        'urbana-champaign'],
                            'location': ['urbana-champaign illinois area']},
                  'U9711': {'college': ['university of illinois at '
                                        'urbana-champaign']}},
  'connections_len': 3,
  'from': 'U9660',
  'inner_global_sum': 6,
  'inner_length_sum': 6,
  'length': 3},
 {'connections': {'U9660': {'college': ['university of illinois at '
  

In [48]:
see_att_main = graph.nodes[main]
print(see_att_main)

false_node = {}

def removeRelatedCharacteristicsInRelationships(key, val, relationships):
    for i, relationship in enumerate(relationships):   
        for node, attributes in relationship['connections'].items():
            print(node)
            if key in attributes: 
                element = attributes[key]
                if val in element: 
                    relationships[i]['connections'][node][key].remove(val)
                

for relationship in relationships:
    if (relationship['inner_global_sum'] == 1) : 
        print('tenemos una caracterízación perfecta')
        unique_key = next(iter(relationship['connections']))
        false_node.update(relationship['connections'][unique_key])
        relationships.remove(relationship)
        key = next(iter(relationship['connections'][unique_key]))
        val = relationship['connections'][unique_key][key][0]
        removeRelatedCharacteristicsInRelationships(key, val, relationships)
        pp.pprint(relationships)
    else :
        print('--- Missing Case ----')
    
    print('false node')
    print(false_node)
        

{'label': 'U9689', 'location': ['cincinnati area'], 'employer': ['general atomics', 'center for plasma and material interactions'], 'college': ['university of illinois at urbana-champaign']}
--- Missing Case ----
false node
{}
--- Missing Case ----
false node
{}
--- Missing Case ----
false node
{}
tenemos una caracterízación perfecta
U9709
U9711
U9657
U9660
U9660
[{'connections': {'U9657': {'college': [],
                            'employer': ['university of illinois at '
                                         'urbana-champaign'],
                            'location': ['urbana-champaign illinois area']},
                  'U9709': {'college': [],
                            'location': ['urbana-champaign illinois area']},
                  'U9711': {'college': []}},
  'connections_len': 3,
  'from': 'U9660',
  'inner_global_sum': 6,
  'inner_length_sum': 6,
  'length': 3},
 {'connections': {'U9660': {'college': [],
                            'employer': ['university of illinois 

In [49]:
# Overall %
def dictOfValues(relationships):
    dict_of_values = {}
    for i, relationship in enumerate(relationships):   
        for node, attributes in relationship['connections'].items():
            for att, values in attributes.items():
                if not att in dict_of_values: dict_of_values[att] = []
                for val in values:
                    if not val in dict_of_values[att]: dict_of_values[att].append(val)
    return dict_of_values
        
    
#def getOverAllAttributesPercentage(attributes, relationships, G, main):
#    attributes_percentage = { item: { 'sum': 0, 'per': 0 } for index, item in enumerate(attributes) }
#    neighbors_pipe = [n for n in G.neighbors(main)]
#    dict_of_values = dictOfValues(attributes, relationships)
#    for i, relationship in enumerate(relationships):
#        for node, attributes in relationship['connections'].items():
#            for att, values in attributes.items():
                
            

In [50]:
# what is this
nodes_taken_out = pd.read_pickle(data_dir + 'mediumRemovedNodes_60percent_of_empty_profile.pickle')

In [51]:
def checkIfInPairsNewPair(triangles_pairs, node_1, node_2):
    if node_1 == node_2: return True
    for triangles_pair in triangles_pairs:
        if node_1 in triangles_pair and node_2 in triangles_pair: return True
    return False

def countTrianglesFromRelatedNodes(node, related_nodes, triangles_pairs):
    triangles_count = 0
    for related_node in related_nodes:  
        #print(f'checking pairs ({node}, {related_node})')
        if not checkIfInPairsNewPair(triangles_pairs, node, related_node): 
            #print(f'found coincidence')
            triangles_pairs.append((node, related_node))
            triangles_count += 0 if len(related_nodes) == 0 else 1
        else :
            continue
    return triangles_count

# Assumes, main node is already idetified
def locateTrianglesInNode(G, neighbors, node):
    triangles_count = 0
    triangles_pairs = []
    for n in neighbors:
        # convert to list in order to be able to use its properties
        neighbors_of_n = [n for n in G.neighbors(n)] 
        neighbors_of_n.remove(node) # ignore main node (node of study)
        related = [f_n for f_n in neighbors_of_n if f_n in neighbors]
        #print(f'neighbor: {n}, related {related}')
        triangles_count += countTrianglesFromRelatedNodes(n, related, triangles_pairs)
                
    return { 'triangles_count': triangles_count, 'triangles_pairs': triangles_pairs }

main = 'U24095'
neighbors = [n for n in graph.neighbors(main)]
print(neighbors)
locateTrianglesInNode(graph, neighbors, main)


['U2734', 'U24113', 'U27287', 'U24087']


{'triangles_count': 0, 'triangles_pairs': []}

In [52]:
count_triangles = 0
count_non_tri_easy = 0
count_non_tri = 0
nodes_with_triangles = []
nodes_one_neighbor = []
nodes_plus_neighbors = []
for node in graph.nodes:
    neighbors = [n for n in graph.neighbors(node)]
    triangles_in_node = locateTrianglesInNode(graph, neighbors, node)
    if triangles_in_node['triangles_count'] == 0 and len(neighbors) > 1: 
        count_non_tri += 1
        nodes_plus_neighbors.append(node)
        print('----SUP--- node: ', node , ' count: ', len(neighbors))
    elif triangles_in_node['triangles_count'] == 0 and len(neighbors) == 1:
        count_non_tri_easy += 1
        print('----EQUAL--- node: ', node)
        nodes_one_neighbor.append(node)
    else:
        count_triangles += 1
        nodes_with_triangles.append(node)
        #print('----TRI--- qty: ', triangles_in_node['triangles_count'])

print('# of nodes w/triangles: ', count_triangles, ' # of same with no: ', count_non_tri_easy, ' others: ', count_non_tri)
print(count_triangles + count_non_tri_easy + count_non_tri)

----EQUAL--- node:  U8804
----SUP--- node:  U9721  count:  2
----SUP--- node:  U2649  count:  2
----EQUAL--- node:  U27759
----EQUAL--- node:  U7310
----EQUAL--- node:  U22859
----EQUAL--- node:  U1417
----SUP--- node:  U24095  count:  4
----EQUAL--- node:  U2620
----EQUAL--- node:  U27776
----EQUAL--- node:  U24549
----EQUAL--- node:  U25632
----EQUAL--- node:  U22031
----EQUAL--- node:  U22071
----EQUAL--- node:  U22018
----EQUAL--- node:  U18925
----EQUAL--- node:  U7136
----EQUAL--- node:  U21990
----SUP--- node:  U3927  count:  2
----EQUAL--- node:  U1381
----EQUAL--- node:  U22087
----EQUAL--- node:  U27804
----EQUAL--- node:  U4447
----EQUAL--- node:  U27679
----EQUAL--- node:  U8029
----EQUAL--- node:  U1075
----EQUAL--- node:  U24259
----EQUAL--- node:  U18564
----EQUAL--- node:  U25619
----EQUAL--- node:  U2661
----EQUAL--- node:  U8000
----EQUAL--- node:  U22015
----SUP--- node:  U9663  count:  2
----EQUAL--- node:  U1441
----EQUAL--- node:  U2714
----EQUAL--- node:  U8051
-

In [53]:
nodes_with_triangles

['U27476',
 'U4665',
 'U1313',
 'U14078',
 'U9628',
 'U25611',
 'U3895',
 'U13005',
 'U5976',
 'U4249',
 'U9140',
 'U16112',
 'U27708',
 'U15284',
 'U18514',
 'U15272',
 'U3933',
 'U15946',
 'U25630',
 'U11562',
 'U15292',
 'U7912',
 'U14498',
 'U24252',
 'U7355',
 'U12969',
 'U2180',
 'U11815',
 'U27582',
 'U27464',
 'U27283',
 'U11575',
 'U14507',
 'U27477',
 'U24143',
 'U24152',
 'U11591',
 'U27585',
 'U27498',
 'U14520',
 'U27300',
 'U9149',
 'U18948',
 'U4600',
 'U4638',
 'U15289',
 'U14005',
 'U24165',
 'U14015',
 'U15329',
 'U24080',
 'U1073',
 'U19856',
 'U4456',
 'U4012',
 'U3958',
 'U24046',
 'U3954',
 'U11564',
 'U15173',
 'U15318',
 'U27460',
 'U11558',
 'U2625',
 'U18954',
 'U3971',
 'U4625',
 'U1045',
 'U16132',
 'U7390',
 'U27661',
 'U8018',
 'U13993',
 'U18914',
 'U14572',
 'U8710',
 'U9660',
 'U2136',
 'U1259',
 'U22825',
 'U10051',
 'U19842',
 'U27657',
 'U15177',
 'U4607',
 'U19913',
 'U27676',
 'U24062',
 'U24077',
 'U27636',
 'U27463',
 'U15168',
 'U27588',
 'U1398

In [54]:
# My Triangle Theory

def searchFromAttributes(G, node_1, node_2, attributes):
    common_att = {}
    node_1_dict = G.nodes[node_1]
    node_2_dict = G.nodes[node_2]
    not_found_att = []
    for att in attributes:
        if att in node_1_dict and att in node_2_dict:
            node_1_att_values = node_1_dict[att]
            node_2_att_values = node_2_dict[att]
            #print(f'found att: {att}: {node_1_att_values} -- {node_2_att_values}')
            # find matches for att between both nodes
            for val in node_1_att_values:
                for val2 in node_2_att_values:
                    prepare_val = ''.join(val.lower().split(' ')) # prevent "errors of hand"
                    prepare_val2 = ''.join(val2.lower().split(' ')) #
                    if (prepare_val == prepare_val2): 
                        if not att in common_att: common_att[att] = []
                        common_att[att].append(val)
        else:
            not_found_att.append(att)
    #if (len(not_found_att) > 0): print('not found att: ', not_found_att)
    return common_att

def getPairRelationship(node_1, node_2, G, attributes):
    connections = searchFromAttributes(G, node_1, node_2, attributes)
    # --- do something else? ---
    return connections

def addValueToNode(G, node, value, attribute):
    node_attributes = G.nodes[node]
    #print(f'node att: {node_attributes}')
    new_values = []
    existant = False
    if attribute in node_attributes:
        new_values = node_attributes[attribute][:]
        prepare_val = ''.join(value.lower().split(' '))
        for val in new_values:
            joined_val = ''.join(val.lower().split(' '))
            if joined_val == prepare_val: 
                existant = True
                break
                
    new_values.append(value)
    #print(f'at node {node} new values: {new_values}')
    if not existant: addAttributes(G, { node: new_values }, attribute)
        
        
node = 'U27476'
neighbors = [n for n in graph.neighbors(node)]
triangles_in_node = locateTrianglesInNode(graph, neighbors, node)
#print(triangles_in_node)

prepare_new_values = {}
for pair in triangles_in_node['triangles_pairs']:
    connections = getPairRelationship(pair[0], pair[1], graph, possible_attributes)
    #print(connections)
    for att, matches in connections.items():
        #print(f'att: { att}, match: { matches}')
        for match in matches:
            addValueToNode(graph, node, match, att)
             

#print(neighbors)
#for neighbor in neighbors:
    # convert to list in order to be able to use its properties
#    first_level_neighbors = [n for n in graph.neighbors(neighbor)] 
#    first_level_neighbors.remove(node) #ignore central node
#    related = [f_n for f_n in first_level_neighbors if f_n in neighbors]
#    print(f'node: {neighbor}, related {related}')

In [55]:
# check properites .. checking issues
print(graph.nodes['U27541'])
print(graph.nodes['U27661'])
print(graph.nodes['U27476'])
# OK theroy, they met in the college, college type of relationship

{'label': 'U27541', 'location': ['greater san diego area'], 'employer': ['uc san diego', 'cisco systems', 'qad inc'], 'college': ['shanghai jiaotong university']}
{'label': 'U27661', 'location': ['san francisco bay area'], 'employer': ['google', 'at&amp;t labs inc.', 'university of michigan'], 'college': ['shanghai jiao tong university']}
{'label': 'U27476', 'college': ['shanghai jiao tong university']}


In [56]:
addAttributes(graph, { 'U27541':  ['lol']}, 'location')
print(graph.nodes['U27541'])
print(graph.nodes['U27661'])
print(graph.nodes['U27476'])

{'label': 'U27541', 'location': ['lol'], 'employer': ['uc san diego', 'cisco systems', 'qad inc'], 'college': ['shanghai jiaotong university']}
{'label': 'U27661', 'location': ['san francisco bay area'], 'employer': ['google', 'at&amp;t labs inc.', 'university of michigan'], 'college': ['shanghai jiao tong university']}
{'label': 'U27476', 'college': ['shanghai jiao tong university']}


In [57]:
well_labeled_nodes = checkWellLabeledNodes(graph)
pipe = [node for node in well_labeled_nodes if node in nodes_with_triangles]
pipe = pipe + [node for node in graph.nodes if node not in pipe]
print(len(pipe))


811


In [58]:
for node in pipe:
    neighbors = [n for n in graph.neighbors(node)]
    triangles_in_node = locateTrianglesInNode(graph, neighbors, node)

    prepare_new_values = {}
    for pair in triangles_in_node['triangles_pairs']:
        connections = getPairRelationship(pair[0], pair[1], graph, possible_attributes)
        for att, matches in connections.items():
            for match in matches:
                addValueToNode(graph, node, match, att)

In [66]:
def extractFromGraph(G, attribute):
    extracted = {}
    for node in G.nodes:
        node_att = G.nodes[node]
        if attribute in node_att: extracted[node] = node_att[attribute]
    return extracted

extracted = extractFromGraph(graph, 'employer')
    

In [69]:
def evaluation_accuracy(groundtruth, pred):
    """    Compute the accuracy of your model.

     The accuracy is the proportion of true results.

    Parameters
    ----------
    groundtruth :  : dict 
       A dict of attributes, either location, employer or college attributes. 
       key is a node, value is a list of attribute values.
    pred : dict 
       A dict of attributes, either location, employer or college attributes. 
       key is a node, value is a list of attribute values. 

    Returns
    -------
    out : float
       Accuracy.
    """
    true_positive_prediction=0   
    for p_key, p_value in pred.items():
        if p_key in groundtruth:
            # if prediction is no attribute values, e.g. [] and so is the groundtruth
            # May happen
            if not p_value and not groundtruth[p_key]:
                true_positive_prediction+=1
            # counts the number of good prediction for node p_key
            # here len(p_value)=1 but we could have tried to predict more values
            true_positive_prediction += len([c for c in p_value if c in groundtruth[p_key]])          
        # no else, should not happen: train and test datasets are consistent
    return true_positive_prediction*100/sum(len(v) for v in pred.values())

print(f'for employer: {evaluation_accuracy(employer,extractFromGraph(graph, "employer"))}')
print(f'for location: {evaluation_accuracy(location,extractFromGraph(graph, "location"))}')
print(f'for college: {evaluation_accuracy(college,extractFromGraph(graph, "college"))}')

for employer: 90.84041548630783
for location: 78.47358121330724
for college: 80.77994428969359
